<a href="https://colab.research.google.com/github/bwsi-hadr/2019-student-final-exercise/blob/master/Logistics_output_files_for_turns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt
!sudo mkdir -p /etc/pki/tls/certs
!sudo cp /etc/ssl/certs/ca-certificates.crt /etc/pki/tls/certs/ca-bundle.crt
try:
  import rasterio
  import rasterio.plot
  import rasterio.merge 
  import rasterio.mask
except:
  !pip install rasterio
  import rasterio
  import rasterio.plot
  import rasterio.merge
  import rasterio.mask
  
try:
  import rasterstats as rs
except:
  !pip install rasterstats  
  import rasterstats as rs
  
try:
  import pyproj
except:
  !pip install pyproj
  import pyproj
  
import networkx as nx
try:
  import osmnx as ox
except:
  # osmnx depends on the system package libspatialindex
  !apt install libspatialindex-dev
  !pip install osmnx
  import osmnx as ox

try: 
  import geopandas as gpd
except: 
  !pip install geopandas 
  import geopandas as gpd
  
try:
  import contextily as ctx 
except:
  # install dependencies for contextily
  !apt install libproj-dev proj-data proj-bin
  !apt install libgeos-dev
  !pip install cython
  !pip install cartopy
  # install contextily
  !pip install contextily==1.0rc1 --no-use-pep517 --no-cache-dir
  import contextily as ctx
  
import fiona
from shapely.geometry import Point, LineString, Polygon
  
import gdal
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import pathlib
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/BWSI-Remote-Sensing/'Final_exercise'/'game2'/

     |████████████████████████████████| 19.7MB 4.9MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 17.9MB 16.4MB/s 
  Created wheel for simplejson: filename=simplejson-3.16.0-cp36-cp36m-linux_x86_64.whl size=114019 sha256=6f7b85402241855401562091472299276fa16f9584721a1703e4ea789366e579
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6613 sha256=8e4fe9d0750fe2a45d6cd70b44d43cf8d3e746cba13b8cb857b35ae71c1f3287
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built simplejson munch
     |████████████████████████████████| 11.2MB 4.7MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt 

In [0]:
global add_supply
def reset_supply():
  global add_supply
  add_supply = pd.DataFrame(columns=['objectid','type','food_water','medicine','fuel'])
def init_supply():
  global add_supply
  add_supply.loc[0,'objectid']=0
  add_supply.fillna(0)
def unpack_powerplants(locations):
  global add_supply
  for i,idx in enumerate(locations):
    add_supply.loc[idx, 'type']='power_plant'
    add_supply.loc[idx, 'food_water']=0
    add_supply.loc[idx, 'medicine']=0
    add_supply.loc[idx, 'fuel']=1200
    #add_supply.loc[idx, 'fuel']=300
    add_supply.loc[idx, 'objectid']=idx
def unpack_airfields(locations):
  global add_supply
  for i,idx in enumerate(locations):
    add_supply.loc[idx, 'type']='airport'
    add_supply.loc[idx, 'food_water']=0
    add_supply.loc[idx, 'medicine']=0
    add_supply.loc[idx, 'fuel']=90
    add_supply.loc[idx, 'objectid']=idx
def add_resources(destind, desttype, foodwater, medicine, fuel):
  global add_supply
  add_supply.loc[destind, 'type'] = desttype
  add_supply = add_supply.fillna(0)
  add_supply.loc[destind, 'food_water'] += foodwater
  add_supply.loc[destind, 'medicine'] += medicine
  add_supply.loc[destind, 'fuel'] += fuel
  add_supply.loc[destind, 'objectid'] = destind
def unpack_large_hospitals(locations):
  global add_supply
  for i,idx in enumerate(locations):
    add_supply.loc[idx, 'type']='hospital_l'
    add_supply.loc[idx, 'food_water']=300
    add_supply.loc[idx, 'medicine']=300
    add_supply.loc[idx, 'fuel']=0
    add_supply.loc[idx, 'objectid']=idx
def unpack_small_hospitals(locations):
  global add_supply
  for i,idx in enumerate(locations):
    add_supply.loc[idx, 'type']='hospital_s'
    add_supply.loc[idx, 'food_water']=150
    add_supply.loc[idx, 'medicine']=150
    add_supply.loc[idx, 'fuel']=0
    add_supply.loc[idx, 'objectid']=idx
def unpack_shelters(locations):
  global add_supply
  for i,idx in enumerate(locations):
    #filew.writerow([idx,'airfield',0,0,300])
    add_supply.loc[idx, 'type']='shelter'
    add_supply.loc[idx, 'food_water']=100
    add_supply.loc[idx, 'medicine']=0
    add_supply.loc[idx, 'fuel']=0
    add_supply.loc[idx, 'objectid']=idx
def unpack_cell_tower(locations):
  global add_supply
  for i,idx in enumerate(locations):
    #filew.writerow([idx,'airfield',0,0,300])
    add_supply.loc[idx, 'type']='cell_tower'
    add_supply.loc[idx, 'food_water']=0
    add_supply.loc[idx, 'medicine']=0
    add_supply.loc[idx, 'fuel']=0
    add_supply.loc[idx, 'objectid']=idx
def distribute_supply(locations,types,foodwater,medicine,fuel):
  global add_suppy
  for i,ix in enumerate(locatins):
    add_supply.loc[idx, 'type'] = types
    add_supply.loc[idx, 'food_water']=foodwater
    add_supply.loc[idx, 'medicine']=medicine
    add_supply.loc[idx, 'fuel']=fuel
    add_supply.loc[idx, 'objectid']=idx
def finalize(exist, path='resources.csv'):
  global add_supply
  if exist:
    existing = pd.DataFrame.from_csv(path)
    new = existing.append(add_supply).fillna(0)
    new.to_csv(path)
    print(new)
  else:
    add_supply.copy().to_csv(path)

In [0]:
global deploy_asset
def reset_deploy():
  global deploy_asset
  deploy_asset = gpd.GeoDataFrame(columns=['objectid','type','geometry'])
  deploy_asset.crs={'init': 'epsg:3785', 'no_defs': True}
def deploy(assetid,assettype,geometry):
  global deploy_asset
  deploy_asset.loc[assetid, 'objectid']=assetid
  deploy_asset.loc[assetid, 'type']=assettype
  deploy_asset.loc[assetid, 'geometry']=geometry
def deploy_distribute(assetidstart,assettype,geometries):
  counter = assetidstart
  for i in geometries:
    deploy_asset.loc[counter,'objectid']=counter
    deploy_asset.loc[counter, 'type']=assettype
    deploy_asset.loc[counter, 'geometry']=i
    counter += 1
def finalize_assets(exist,path='assets.shp'):
  global deploy_asset
  if exist:
    existing = gpd.GeoDataFrame.from_file(path)
    new = existing.append(deploy_asset).fillna(0)
    print(new)
    new.to_file(path)
  else:
    deploy_asset.copy().to_file(path)

In [0]:
def main():
  print("Welcome. Assets or Resources (a/r)?")
  choice = input()
  if choice == 'r':
    print('Enter destination ids (object id or id), singular or list form [x y z]:')
    dests = input()
    if dests[0] == '[':
      ids = dests[1:-1].split(',')
    else:
      ids = [int(dests)]
    print('Destination ids: ', ids)
    print('What type is each destination?')
    desttypes = []
    for i in ids:
      print(i,':',sep='', end=' ')
      desttypes += [input()]
    print('How much food/water?')
    foodwater = []
    for idx, i in enumerate(ids):
      print(i,' ',desttypes[idx],':',sep='', end=' ')
      foodwater += [int(input())]
    print('How much medicine?')
    medicine = []
    for idx, i in enumerate(ids):
      print(i,' ',desttypes[idx],':',sep='', end=' ')
      medicine += [int(input())]
    print('How much fuel?')
    fuel = []
    for idx, i in enumerate(ids):
      print(i,' ',desttypes[idx],':',sep='', end=' ')
      fuel += [int(input())]
    print('ind,id,type,f/w,meds,fuel')
    for i in list(zip(range(len(ids)),ids,desttypes,foodwater,medicine,fuel)):
      print(*i)
    while True:
      edit = int(input('Edit index? -1 to finish: '))
      if edit == -1:
        break
      else:
        i = input('id? -1 to keep: ')
        if i == '-1':
          i = int(i)
          i = ids[edit]
        d = input('type? -1 to keep: ')
        if d == '-1':
          d = int(d)
          d = desttypes[edit]
        fw = input('food/water? -1 to keep: ')
        if fw == '-1':
          fw = int(fw)
          fw = foodwater[edit]
        else:
          fw = int(fw)
        med = input('medicine? -1 to keep: ')
        if med == '-1':
          med = int(med)
          med = medicine[edit]
        else:
          med = int(med)
        f = input('fuel? -1 to keep: ')
        if f == '-1':
          f = int(f)
          f = fuel[edit]
        else:
          f = int(f)
        ids[edit] = i
        desttypes[edit]=d
        foodwater[edit]=fw
        medicine[edit]=med
        fuel[edit]=f
        print('ind,id,type,f/w,meds,fuel')
        for i in list(zip(range(len(ids)),ids,desttypes,foodwater,medicine,fuel)):
          print(i)
    for idx, i in enumerate(ids):
      add_resources(i,desttypes[idx],foodwater[idx],medicine[idx],fuel[idx])
    print(add_supply)
  elif choice == 'a':
    print('Enter the first asset id')
    id1 = int(input())
    idt = id1
    print("What is the asset type?")
    typ = input()
    print("How many assets do you want?")
    cnt = int(input())
    print("Put a list of coordinates of the asset, as in [x1,y1,x2,y2...]")
    coords = input().replace('[', '').replace(']', '')
    geometries = []
    coords = coords.split(',')
    for i in range(0, len(coords), 2):
      geometries.append(Point(float(coords[i]), float(coords[i+1])))
    ids = [id1+i for i in range(cnt)]
    desttypes = [typ for i in range(cnt)]


    print('ind,id,type,geometry')
    for i in list(zip(range(len(ids)),ids,desttypes,geometries)):
      print(str(i[:-1]) + " " + str(i[-1]))

    for idx, i in enumerate(ids):
      deploy(i,desttypes[idx],geometries[idx])  
  else:
    print('Error: incorrect input')

In [0]:
reset_supply()
reset_deploy()

In [0]:
main()

Welcome. Assets or Resources (a/r)?
r
Enter destination ids (object id or id), singular or list form [x y z]:
[338.0, 564.0, 397.0, 233.0, 642.0, 466.0, 21.0, 481.0, 512.0, 546.0]
Destination ids:  ['338.0', ' 564.0', ' 397.0', ' 233.0', ' 642.0', ' 466.0', ' 21.0', ' 481.0', ' 512.0', ' 546.0']
What type is each destination?
338.0: power_plant
 564.0: power_plant
 397.0: power_plant
 233.0: power_plant
 642.0: power_plant
 466.0: power_plant
 21.0: power_plant
 481.0: power_plant
 512.0: power_plant
 546.0: power_plant
How much food/water?
338.0 power_plant: power_plant


ValueError: ignored

In [0]:
deploy_asset

,objectid,type,geometry
1000000,1000000,generator_l,POINT (-7961984.714670815 5131569.318735286)
573957834,573957834,generator,POINT (63634.6546 35.56)
68454,68454,generator,POINT (70 70)
68596825,68596825,generator,POINT (6364 534)
68596826,68596826,generator,POINT (636 4)


In [0]:
reset_supply()

Turn 0:

In [0]:
reset_supply()
unpack_powerplants([338.0, 564.0, 397.0, 233.0, 642.0, 466.0, 21.0, 481.0, 512.0, 546.0, 486.0, 658.0, 459.0, 460.0, 577.0, 180.0, 59.0, 39.0, 219.0, 526.0])

In [0]:
reset_supply()

In [0]:
unpack_powerplants([338.0, 564.0, 397.0, 233.0, 642.0, 466.0, 21.0, 481.0, 512.0, 546.0,486.0, 658.0, 459.0, 460.0, 577.0, 180.0, 59.0, 39.0, 219.0, 526.0,468.0, 337.0, 550.0, 371.0, 410.0, 92.0, 453.0, 401.0]) 
#RESPONSE TURN 0

In [0]:
unpack_airfields([7519, 7661, 7690, 7650, 7695,])

In [0]:
add_resources(7695,'airfield',150000,150000,37500)

In [20]:
add_supply #check order

,objectid,type,food_water,medicine,fuel
338.0,338,power_plant,0,0,1200
564.0,564,power_plant,0,0,1200
397.0,397,power_plant,0,0,1200
233.0,233,power_plant,0,0,1200
642.0,642,power_plant,0,0,1200
466.0,466,power_plant,0,0,1200
21.0,21,power_plant,0,0,1200
481.0,481,power_plant,0,0,1200
512.0,512,power_plant,0,0,1200
546.0,546,power_plant,0,0,1200


In [0]:
finalize(False) #False if file doesn't already exist

In [0]:
reset_supply()

In [0]:
unpack_powerplants([468.0, 337.0, 550.0, 371.0, 410.0, 92.0, 453.0, 401.0, 457.0, 83.0, 583.0,310.0, 100.0])

In [0]:
add_supply

,objectid,type,food_water,medicine,fuel
468.0,468,power_plant,0,0,300
337.0,337,power_plant,0,0,300
550.0,550,power_plant,0,0,300
371.0,371,power_plant,0,0,300
410.0,410,power_plant,0,0,300
92.0,92,power_plant,0,0,300
453.0,453,power_plant,0,0,300
401.0,401,power_plant,0,0,300


In [0]:
#run for turn 3
finalize(False)

In [0]:
finalize(True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:78: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


        objectid            type  food_water  medicine   fuel
338.0      338.0     power_plant           0         0    300
564.0      564.0     power_plant           0         0    300
397.0      397.0     power_plant           0         0    300
233.0      233.0     power_plant           0         0    300
642.0      642.0     power_plant           0         0    300
466.0      466.0     power_plant           0         0    300
21.0        21.0     power_plant           0         0    300
481.0      481.0     power_plant           0         0    300
512.0      512.0     power_plant           0         0    300
546.0      546.0     power_plant           0         0    300
486.0      486.0     power_plant           0         0    300
658.0      658.0     power_plant           0         0    300
459.0      459.0     power_plant           0         0    300
460.0      460.0     power_plant           0         0    300
577.0      577.0     power_plant           0         0    300
180.0   

In [0]:
reset_supply()

In [0]:
unpack_powerplants([360.0,73.0, 274.0, 366.0, 28.0, 194.0, 205.0, 518.0, 3.0, 252.0, 278.0, 110.0, 37.0, 159.0, 595.0, 476.0, 595.0, 476.0, 595.0, 476.0, 595.0, 476.0, 595.0,476.0])

In [0]:
finalize(False)